## Component-Wise Extraction & Labeling of UNSW PCAP files 

This approach comprises of extracting pcap file features utilizing developed pcap parser. Then utilizing it for comparison and labelling. Prominent steps for this approach are:
1. Extract Pcap file features into csv using developed parser. 
2. Perform labelling by comparing pre-processed UNSW CSV file and pcap parsed file (in .csv format).This approach compares both file using 8 distinct features which are :
> a)  Start Time(*stime*) 
>
> b)  Last Time(*ltime*)
>
> c)  Source IP Address(*srcip*) 
>
> d)  Destination IP Address(*dstip*)
>
> e)  Destination Port Number(*dsport*)
>
> f)  Source Port Numbe(*sport*)
>
> g)  Source Time to Live Value (*sttl*)
>
> h)  Transaction Protocol(*protocol_m*)
            
3. Transform data into byte-wise features for machine learning models


In [ ]:
import sys
sys.path.append("C:\\Users\\yasir.ali\\Project\\Payload-Byte")
## append your project path if its not already present

In [ ]:
import hashlib as hl
import csv
from os import close
import numpy as np
import logging
import argparse
import sys
from datetime import datetime
import socket
import math
import random
import pandas as pd
import scapy.all as sc
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler 
import logging
import sys
from functions.Optimized_Parser_Labelling import *
from functions.Pipeline import *

## Parsing Pcap Files into CSV format

In [ ]:
root = logging.getLogger()
root.setLevel(logging.DEBUG)

In [ ]:
out_file='G:/UNSW_result/pcap_file_csv_parser/17-2-1015/'
pcap='F:/UNSW_pcap/17-2-2015/' ## Change directory for 22-01-2015
pcap_file=[]
for x in range(1,28): ## Starting & Ending values for the files
    pcap_file.append(pcap+str(x)+".pcap")

In [ ]:
pcap_parser(pcap_file,out_file,1)

## Labelling 

In [ ]:
df_p=pd.read_csv("E:/UNSW-NB15 Dataset/UNSW-NB15-CSV-Files/Preprocessed-CSV/UNSW-NB15_processed.csv")
df_p=df_p[['stime','ltime','srcip','dstip','dsport','sport','sttl','proto','dur','attack_cat','label']]
df_p=df_p.sort_values(by='stime')

In [ ]:
pcap='G:/UNSW_result/pcap_file_csv_parser/'
pcap_csv=[]
for x in range(1,53): ## Starting & Ending values for the files
    pcap_csv.append(pcap+"pcap_csv_"+str(x)+".csv")
UNSW_csv="E:/UNSW-NB15 Dataset/UNSW-NB15-CSV-Files/Preprocessed-CSV/UNSW-NB15_processed_v4.csv"
output_file="G:/UNSW_result/Labelled_pcap_file/22-1-2015/"

In [ ]:
file_num= 1
label(pcap_csv,UNSW_csv,output_file,file_num)

## Reading and Combining All Labelled Files

In [ ]:
## For 22-1-2015 Day
label_csv="G:/UNSW_result/Labelled_pcap_file/22-1-2015/"
in_file=[]
for x in range(1,53): ## Starting & Ending values for the files
    in_file.append(label_csv+"labelled_pcap_csv_"+str(x)+".csv")


In [ ]:
## For 17-2-2015 Day
a=[]
label_csv="G:/UNSW_result/Labelled_pcap_file/17-2-2015/"
for x in range(1,27): ## Starting & Ending values for the files
    a.append(label_csv+"labelled_pcap_csv_"+str(x)+".csv")
    
a=a+in_file
out_path="G:/UNSW_result/Labelled_pcap_file/"

In [ ]:
df_payload=combine(a,out_path)
df_payload.shape

## Pre-processing Combined Payload File

In [ ]:
df_payload.drop_duplicates(inplace=True)
df_payload.drop(df_payload[df_payload.payload.isnull()].index,inplace=True)

In [ ]:
df_payload['payload_int'] = df_payload['payload'].apply(int, base=16)
df_payload.drop(df_payload[df_payload.payload_int==0].index,inplace=True)

In [ ]:
df_payload.pop('payload_int')
df_payload.sort_values(by=['stime','frame_num'],inplace=True,ignore_index=True)

In [ ]:
df_payload.sttl=df_payload.sttl.astype('int32')
df_payload.dsport=df_payload.dsport.astype('int32')
df_payload.sport=df_payload.sport.astype('int32')
df_payload.total_len=df_payload.total_len.astype('int32')
df_payload.to_csv("G:/UNSW_result/Labelled_pcap_file/combined_labelled_cleaned_sorted_pcap_csv.csv",index=False)

## Transformation into Byte-wise Features

In [ ]:
## Undersampling Data instances
dict={ 'generic':17580,
'exploits':13992,
'fuzzers'  : 12722,
'reconnaissance': 7562,
'dos'  : 3397,
'backdoor' :   1239,
'analysis' :  1208,
'shellcode': 1088,
'normal': 21000,
'worms':  93
}

In [ ]:
rus = RandomUnderSampler(random_state=42,sampling_strategy=dict)
X_res, y_res = rus.fit_resample(df_payload.iloc[:,:-2], df_payload.iloc[:,-2])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(X_res['protocol_m'])
X_res['protocol']=label

X_res.drop("protocol_m", axis=1, inplace=True)
X_res.drop("protocol_s", axis=1, inplace=True)
X_res.drop("frame_num", axis=1, inplace=True)

X_res['attack_cat']=y_res

In [ ]:
X_tr,Ytrain =payload_to_bytes(X_res,1500)

In [ ]:
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,6])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,7])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,5])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,9])))

In [ ]:
name=[]
for x in range(1,1501):
    name.append("payload_byte_"+str(x))
name.append("ttl")
name.append("total_len")
name.append("protocol")
name.append("t_delta")

In [ ]:
df = pd.DataFrame(X_tr, columns=name)
df['label']=Ytrain

In [ ]:
df.to_csv("F:/Converted_data.csv",index=False)